In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/HEURISTICS/Uganda_predictions')

In [ ]:
pip install requests beautifulsoup4 rasterio rasterstats fiona geopandas earthpy -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.3 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiLineString
# Import the CustomIcon class
from folium.features import CustomIcon
import datetime
import os
import glob
import rasterio
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import matplotlib.dates as md


In [ ]:
mining_data = pd.read_excel("/content/drive/MyDrive/HEURISTICS/Uganda_predictions/new_mining_sites/FGD_MR_GPS.xlsx")

In [ ]:
mining_data

,FGD_type,FGD_ID,District,Mining_site_GPS,FGD_GPS
0,Women,UGA0001,Kaabong,3.53269N;\n34.27366E,3.3157N; 34.1618E
1,Women,UGA0001,Kaabong,3.53269N;\n34.27366E,3.3157N; 34.1618E
2,Women,UGA0001,Kaabong,3.53269N;\n34.27366E,3.3157N; 34.1618E
3,Women,UGA0001,Kaabong,3.53269N;\n34.27366E,3.3157N; 34.1618E
4,Women,UGA0001,Kaabong,3.53269N;\n34.27366E,3.3157N; 34.1618E
...,...,...,...,...,...
162,Mixed,UGA0014,Amudat,1.925211N;\n34.954624E,1.92511N 34.954624E
163,Mixed,UGA0014,Amudat,1.925211N;\n34.954624E,1.92511N 34.954624E
164,Mixed,UGA0014,Amudat,1.925211N;\n34.954624E,1.92511N 34.954624E
165,Mixed,UGA0014,Amudat,1.925211N;\n34.954624E,1.92511N 34.954624E


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
df = mining_data.copy()
# Remove 'N' and 'E' characters and split coordinates
df[['Latitude', 'Longitude']] = df['Mining_site_GPS'].str.replace('[NE]', '', regex=True).str.split(';', expand=True)

# Convert Latitude and Longitude to float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# Drop the original 'Coordinates' column
#df.drop(columns=['Coordinates'], inplace=True)

# Create geometry column from coordinates
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Save as GeoJSON file
gdf.to_file("/content/drive/MyDrive/HEURISTICS/Uganda_predictions/new_mining_sites/field_gps.geojson", driver="GeoJSON")

In [ ]:
# Convert UTM coordinates to latitude and longitude
gps_coordinates = values.get('GPS Coordinates', '')
if gps_coordinates:
    x, y = map(float, gps_coordinates.split())
    # Define the UTM and geographic coordinate systems
    utm = pyproj.Proj(proj='utm', zone=36, ellps='WGS84')
    wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')
    # Convert UTM coordinates to latitude and longitude
    longitude, latitude = pyproj.transform(utm, wgs84, x, y)
    values['Latitude'] = latitude
    values['Longitude'] = longitude
      # Get location name from Google using reverse geocoding
    #location_name = get_location_name(values['Latitude'], values['Longitude'])
    #values['Location'] = location_name

In [ ]:
# Initialize the Earth Engine module
import ee
ee.Authenticate()
ee.Initialize(project='ee-janet')

# Define the dataset
dataset = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')

# Define visualization parameters
visualization = {
    'bands': ['seasonality'],
    'min': 0.0,
    'max': 12.0,
    'palette': ['red','blue']
}

# Define the area of interest (AOI)
aoi = ee.FeatureCollection('projects/ee-janet/assets/Uganda/karamoja')
aoi = aoi.geometry()

# Center the map on the AOI
center_coords = aoi.centroid().coordinates().reverse().getInfo()
map = folium.Map(location=center_coords, zoom_start=11)

# Add the dataset layer to the map
mapid = dataset.getMapId(visualization)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='seasonality',
).add_to(map)

# Display the map
map

In [ ]:
uganda_data = gpd.read_file('/content/drive/MyDrive/HEURISTICS/Uganda_predictions/UGA_adm/UGA_adm4.shp')

In [ ]:
uganda_data.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,ID_3,NAME_3,ID_4,NAME_4,VARNAME_4,TYPE_4,ENGTYPE_4,geometry
0,239,UGA,Uganda,1,Adjumani,1,East Moyo,1,Adjumani Tc,1,Biyaya,None,Parish,Parish,"POLYGON ((31.81690 3.38219, 31.81677 3.38176, ..."
1,239,UGA,Uganda,1,Adjumani,1,East Moyo,1,Adjumani Tc,2,Central,None,Parish,Parish,"POLYGON ((31.80799 3.37267, 31.80773 3.37231, ..."
2,239,UGA,Uganda,1,Adjumani,1,East Moyo,1,Adjumani Tc,3,Cesia,None,Parish,Parish,"POLYGON ((31.78006 3.36726, 31.77676 3.36743, ..."
3,239,UGA,Uganda,1,Adjumani,1,East Moyo,2,Adropi,4,Esia,None,Parish,Parish,"POLYGON ((31.71377 3.25949, 31.71348 3.26290, ..."
4,239,UGA,Uganda,1,Adjumani,1,East Moyo,2,Adropi,5,Jihwa,None,Parish,Parish,"POLYGON ((31.78742 3.46698, 31.78643 3.46754, ..."


In [ ]:
uganda_data.columns

Index(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'NAME_1', 'ID_2', 'NAME_2', 'ID_3',
       'NAME_3', 'ID_4', 'NAME_4', 'VARNAME_4', 'TYPE_4', 'ENGTYPE_4',
       'geometry'],
      dtype='object')

In [ ]:
import pandas as pd

miners = ['morulem gold miners association','artisanal mining group','sogdek miners association','ticodep (tapac integrated community development organization)',
          'lotakero artisanal',"rupa community development trust (rucodet)","moruita artisanal groups (individuals working in an informal way)",
          "acerer artisanal groups (individuals working in an informal way)","koriitu community development trust (it is also a communal land association)",
          "local artisanal individuals in lopedo","rakai resources","oks mining company ltd-international/russian", "tororo cement-ugandan",
          "sunbelt-chinese","sunbird resources ltd-international","kiteelore enterprise lt-local from moroto","jinja marble", "agro-mechanized",
          "west international holding limited-part of china west cement limited",
          "ateker cement, uganda development cooporation and savanah mines","ateker cement, uganda development cooporation and savanah mines", "african energy (national)"]

village = [ 'nan','chepkararat','nan','nan','nan','nan','nan','nan','lapedo','nan','nan','nan','kosiroi','nan','nan','nan','nan','nadunget','nan','nan','lopedo','nan']
mining_scale = ["small","small","small","small","small","small","small","small","small","small","large","large","large","large","large","large","large","large","large",
                "large","large","large"]
subcounty = ['morulem','karita','tapac','tapac','rupa',"rupa","moruita","moruita","loyoro","loyoro",'morulem',"karita",
    'tapac','rupa','rupa','rupa','rupa','rupa','nadunget',"loyoro",'nadunget','loyoro']

district = ['abim','amudat','moroto','moroto',' moroto',"moroto","nakapiripirit","nakapiripirit","kaabong", "kaabong",'abim',"amudat",
            'moroto','moroto','moroto','moroto','moroto','moroto','moroto',"kaabong",'moroto','kaabong']
minerals =['gold','gold','limestone','limestone',"gold,marble,limestone","gold,marble,limestone","gold","gold",
           "gold and limestone", "gold","gold and also does exploration","gold", "limestone and marble","marble and gold","marble",
           "limestone","limestone","marble and gold","limestone and establishing cement factory, sand/silica","exploration, limestone, planning to establish cement factory in moroto",
           "exploration, limestone, planning to establish cement factory in moroto",'gold']

status = ['active','active','active','active',"active","active","active","active","active","active","active","active","active for marble", "active","active",
          "active","active","active", "200 acres for factory, not yet active",'not indicated','not indicated','active']
# Create a DataFrame from the list
new_mining_data = pd.DataFrame({'miner_names': miners, 'village': village, 'mining_scale':mining_scale,'subcounty':subcounty,
                   'district':district,'status':status,'minerals_names': minerals})

new_mining_data

,miner_names,village,mining_scale,subcounty,district,status,minerals_names
0,morulem gold miners association,nan,small,morulem,abim,active,gold
1,artisanal mining group,chepkararat,small,karita,amudat,active,gold
2,sogdek miners association,nan,small,tapac,moroto,active,limestone
3,ticodep (tapac integrated community developmen...,nan,small,tapac,moroto,active,limestone
4,lotakero artisanal,nan,small,rupa,moroto,active,"gold,marble,limestone"
5,rupa community development trust (rucodet),nan,small,rupa,moroto,active,"gold,marble,limestone"
6,moruita artisanal groups (individuals working ...,nan,small,moruita,nakapiripirit,active,gold
7,acerer artisanal groups (individuals working i...,nan,small,moruita,nakapiripirit,active,gold
8,koriitu community development trust (it is als...,lapedo,small,loyoro,kaabong,active,gold and limestone
9,local artisanal individuals in lopedo,nan,small,loyoro,kaabong,active,gold


In [ ]:
location_names = [
    'kosiroi', 'nakabat', 'naput', 'nanyidik', 'lotorongor', 'tapac', 'nakiloro', 'nakibat',
    'rata', 'katakekile', 'kambizi', 'nosing', 'loyoro', 'lodiko', 'lopedot','acerere',
    'chepkararat', 'morulem', 'abim', 'karita', 'amudat', 'nabilatuk', 'karenga', 'lolachet',
    'tiira', 'busia', 'buhweju', 'mubende', 'nakapiripirit', 'kaabong', 'kamun', 'rupa'
]
minerals =['limestone','gold','marble','marble','marble','gold','gold','gold','marble','marble'
,'construction minerals','gold','gold','gold','gold' ,'gold',
           'gold','gold','gold','gold','gold','gold','gold','silicasan','gold','gold','gold',
           'gold','gold','gold','gold','gold']
status = ['active']
# Create a DataFrame from the list
old_mining_data = pd.DataFrame({'Location Names': location_names, 'minerals names': minerals})

old_mining_data

,Location Names,minerals names
0,kosiroi,limestone
1,nakabat,gold
2,naput,marble
3,nanyidik,marble
4,lotorongor,marble
5,tapac,gold
6,nakiloro,gold
7,nakibat,gold
8,rata,marble
9,katakekile,marble


In [ ]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

def geocode_locations(data_frame, location_column):
    """
    Geocode locations in a DataFrame and create a GeoDataFrame.

    Parameters:
    - data_frame (pd.DataFrame): The DataFrame containing location names.
    - location_column (str): The name of the column containing location names.

    Returns:
    - gpd.GeoDataFrame: GeoDataFrame with latitude and longitude columns added.
    """

    # Initialize the geocoder
    geolocator = Nominatim(user_agent="geocoder")

    # List of place names
    place_names = list(data_frame[location_column])

    # Create lists to store latitude and longitude
    latitudes = []
    longitudes = []

    # Geocode the place names and update the latitude and longitude lists
    for place_name in place_names:
        try:
            location = geolocator.geocode(place_name)
            if location:
                latitudes.append(location.latitude)
                longitudes.append(location.longitude)
            else:
                latitudes.append(None)
                longitudes.append(None)
        except Exception as e:
            latitudes.append(None)
            longitudes.append(None)

    # Append latitude and longitude columns to the DataFrame
    data_frame['Latitude'] = latitudes
    data_frame['Longitude'] = longitudes

    # Create a GeoDataFrame
    geometry = [Point(xy) for xy in zip(data_frame.Longitude, data_frame.Latitude)]
    location_df = gpd.GeoDataFrame(data_frame, geometry=geometry)

    # Filter out rows with missing latitude values
    subset_location_df = location_df[~location_df['Latitude'].isnull()]

    return subset_location_df


result_geo_df = geocode_locations(new_mining_data, 'subcounty')


In [ ]:
results_old_mining_data = geocode_locations(old_mining_data,'Location Names')

In [ ]:
result_geo_df.columns

Index(['miner_names', 'village', 'mining_scale', 'subcounty', 'district',
       'status', 'minerals_names', 'Latitude', 'Longitude', 'geometry'],
      dtype='object')

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geocoder")

# List of place names
place_names = list(old_mining_data['Location Names'])

# Create lists to store latitude and longitude
latitudes = []
longitudes = []

# Geocode the place names and update the latitude and longitude lists
for place_name in place_names:
    try:
        location = geolocator.geocode(place_name)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)

# Append latitude and longitude columns to the DataFrame
old_mining_data['Latitude'] = latitudes
old_mining_data['Longitude'] = longitudes


# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(old_mining_data.Longitude, old_mining_data.Latitude)]
location_df = gpd.GeoDataFrame(old_mining_data, geometry=geometry)

subset_location_df = location_df[~location_df['Latitude'].isnull()]

In [ ]:
result_geo_df.to_file('new_mining_sites/new_mining_sites_gordon.shp')
result_geo_df.to_file('/content/drive/MyDrive/HEURISTICS/Uganda_predictions/new_mining_sites/new_mining_sites_gordon.geojson')

<ipython-input-14-7d9c51f19150>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result_geo_df.to_file('new_mining_sites/new_mining_sites_gordon.shp')


In [ ]:
osm_data = gpd.read_file('uganda-latest-free.shp/gis_osm_landuse_a_free_1.shp')
#osm_waterways = gpd.read_file('shapefiles/karamoja_waterways.geojson')
osm_water = gpd.read_file('shapefiles/karamoja_waterways.geojson')
osm_landuse = gpd.read_file('data/karamoja_landuse_up.shp')

In [ ]:
#read the other data
mining_one = gpd.read_file('new_mining_sites/landuse_mining_areas_v1.shp')
mining_two = gpd.read_file('new_mining_sites/up_mining_31.shp')
mining_three = gpd.read_file('new_mining_sites/updated_mining_93.shp')
#cleaning
mining_one[['Name','fclass', 'geometry']]
# Assuming mining_one is your GeoDataFrame
mining_one['Name'] = mining_one['Name'].fillna(mining_one['fclass'])
mining_one = mining_one[['Name', 'geometry']]
mining_two = mining_two[['Name','geometry']]
mining_three = mining_three[['Name','geometry']]

In [ ]:
concatenated_df = pd.concat([mining_two, mining_three], ignore_index=True)

In [ ]:
concatenated_df.shape

(124, 2)

In [ ]:
concatenated_df['scale']=[]

ValueError: Length of values (0) does not match length of index (124)

In [ ]:
# Create an empty 'scale' column
concatenated_df['scale'] = ''
concatenated_df['mineral'] = ''

for _, row in concatenated_df.iterrows():
    mineral = row['Name']
    scale = row['scale']
    split_name = mineral.split('-') if '-' in mineral else ['', '']

    # Assuming you want to append the second part of the split_name to the 'scale' column
    concatenated_df.at[_, 'scale'] = split_name[1]
    concatenated_df.at[_, 'mineral'] = split_name[0]

# Drop the 'Name' column
concatenated_df.drop(columns=['Name'], inplace=True)

In [ ]:
concatenated_df.to_file('/content/drive/MyDrive/HEURISTICS/Uganda_predictions/data/rs_mining_sires_ug.shp')

In [ ]:
import folium
import geopandas as gpd
from shapely.geometry import Point

crs = 'EPSG:4326'
m = folium.Map(location=[31.40955, 0.51543], zoom_start=11)  # Adjust the location and zoom level as needed


# Add the points to the map
for idx, row in result_geo_df.iterrows():
    folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],
        popup=row['minerals_names'],  # Customize the popup content if needed
    ).add_to(m)

# Display the map
m


In [ ]:
common_crs ='EPSG:4326'
# Define the GeoJSON file paths
level_4_boundaries = gpd.read_file('shapefiles/karamoja_level_4_boundaries/karamoja_level_4_bnd.shp').to_crs(common_crs)
karamoja_roads = gpd.read_file('shapefiles/karamoja_roads.geojson').to_crs(common_crs)
waterways = gpd.read_file('shapefiles/karamoja_waterways.geojson').to_crs(common_crs)

In [ ]:
mining_one.Name.value_counts()

residential                       659
forest                             94
farmland                           54
scrub                              37
heath                              34
                                 ... 
Nasiyon                             1
a good patch of land :)             1
Apeitolim                           1
military                            1
UPDF Barracks Nataba - Disused      1
Name: Name, Length: 66, dtype: int64

In [ ]:
karamoja_roads.ref.value_counts()

B30203    1
E368      1
Name: ref, dtype: int64

### Mapping the mineral map in Uganda

This code includes the mapping of the areas that were proposed by stakeholders. Hopefully when we get the right names we can clearly see them from the maps

In [ ]:
# Create a Folium map with a title
m = folium.Map(location=[31.40955, 0.51543], zoom_start=8, tiles='Stamen Terrain',  attr='Your attribution here', control_scale=True) #tiles='Stamen Terrain'
title = '<h3 align="center" style="font-size:100px"><b>Uganda minerals mapping</b></h3>'
title_html = folium.Html(title, script=True)
title_popup = folium.Popup(title_html, max_width=2650)

common_crs ='EPSG:4326'
# Define the GeoJSON file paths
level_4_boundaries = gpd.read_file('shapefiles/karamoja_level_4_boundaries/karamoja_level_4_bnd.shp').to_crs(common_crs)
karamoja_roads = gpd.read_file('shapefiles/karamoja_roads.geojson').to_crs(common_crs)
waterways = gpd.read_file('shapefiles/karamoja_waterways.geojson').to_crs(common_crs)

#read the other data
mining_one = gpd.read_file('new_mining_sites/landuse_mining_areas_v1.shp')
mining_two = gpd.read_file('new_mining_sites/up_mining_31.shp')
mining_three = gpd.read_file('new_mining_sites/updated_mining_93.shp')

#cleaning
mining_one[['Name','fclass', 'geometry']]
# Assuming mining_one is your GeoDataFrame
mining_one['Name'] = mining_one['Name'].fillna(mining_one['fclass'])
mining_one = mining_one[['Name', 'geometry']]
mining_two = mining_two[['Name','geometry']]
mining_three = mining_three[['Name','geometry']]

major_roads = ['motorway','trunk','primary','secondary','tertiary','motorway_link','trunk_link',
               'primary_link','secondary_link','tertiary_link']
minor_roads = ['unclassified','residential','living_street','pedestrian','busway','service',
               'track','track_grade1','track_grade2','track_grade3','track_grade4','track_grade5',
               'bridleway','cycleway','footway','path','steps','unknown']

major_rivers = ['river']
minor_rivers = ['stream','canal','drain']

# Add karamoja region as a GeoJSON layer with a custom color

karamoja_geojson = folium.GeoJson(level_4_boundaries, name='Karamoja Region', style_function=lambda x: {
    'fillColor': '#D0BFFF',  # Change the color here
    'color': '#D0BFFF',
    'weight': 1,
    'fillOpacity': 0.6
}).add_to(m)

# Add corridors as MultiLineString with a name for the legend and a custom color
roads_json = karamoja_roads.to_json()
folium.features.GeoJson(roads_json, name='Roads', style_function=lambda x: {
    'color': '#862B0D',  # Change the color here
    'weight': 2
}).add_to(m)


# Add corridors  proposed by stakeholders as MultiLineString with a name for the legend and custom colors
for _, row in karamoja_roads.iterrows():
    if row['fclass'] is not None:
        name = row['fclass'].lower()
    #else:
      #  name = 'unknown'
    #color = 'red'  # Default color

    if name in major_roads:
        color = '#000000'
    elif name in minor_roads:
        color = '#000000'

    # Check if the geometry is a MultiLineString
    if isinstance(row['geometry'], MultiLineString):
        for line_string in row['geometry'].geoms:
            # Create a PolyLine with the specified color
            line = folium.PolyLine(
                locations=line_string.coords,
                color=color,
                weight=2,
                name=name
            )

            # Add the line to the map
            line.add_to(m)
# Add villages as small yellow circles with a name for the legend
for _, row in mining_one.iterrows():
    centroid = row['geometry'].centroid
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=3,
        color='#F1C93B',
        fill=True,
        fill_color='#F1C93B',
        popup=row['Name']  # Information to display on click
    ).add_to(m)


# # Add villages as small yellow circles with a name for the legend
# for _, row in mining_one.iterrows():
#     folium.CircleMarker(
#         location=[row['geometry'].y, row['geometry'].x],
#         radius=3,
#         color='#F1C93B',
#         fill=True,
#         fill_color='#F1C93B',
#         popup=row['Name']  # Information to display on click
#     ).add_to(m)

# Add resting points as small green circles on top of the corridors with a name for the legend
for _, row in mining_two.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=3,
        color='#279EFF',
        fill=True,
        fill_color='#279EFF',
        popup=row['Name']  # Information to display on click
    ).add_to(m)
# Add resting points as small green circles on top of the corridors with a name for the legend
for _, row in mining_three.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=3,
        color='#279EFF',
        fill=True,
        fill_color='#279EFF',
        popup=row['Name']  # Information to display on click
    ).add_to(m)


# Update the legend HTML to include the radio icon and description
legend_html = '''
<div style="position: fixed;
             bottom: 30%; left: 30%; transform: translate(-50%, -50%);
             width: 180px; height: 250px;
             border:2px solid grey; z-index:9999; font-size:14px;
             background-color: white;
             opacity: 0.8;">
    <b>Legend</b><br>
    <i style="background: #279EFF; width: 10px; height: 10px; display: inline-block;"></i> Resting Points<br>
    <i style="background: #F1C93B; width: 10px; height: 10px; display: inline-block;"></i> Villages<br>
    <i style="background: #A7D397; width: 10px; height: 10px; display: inline-block;"></i> Ponds<br>
     <i style="background: #C70039; width: 10px; height: 10px; display: inline-block;"></i> Radio stations <br>
    <i style="background: #862B0D; width: 10px; height: 10px; display: inline-block;"></i> Corridors<br>
    <i style="background: #D0BFFF; width: 10px; height: 10px; display: inline-block;"></i> Ferlo Region<br>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))


# Add a layer control to the map
folium.LayerControl().add_to(m)

# Add the title to the map
title_popup.add_to(m)

# Save the map to an HTML file
m.save('ikon_deployment_map_option_one.html')
m


This code does not include the mapping of the areas that were proposed by stakeholders.

In [ ]:
# Create a Folium map with a title
m = folium.Map(location=[14.4974, -14.4524], zoom_start=6, tiles='OpenStreetMap', control_scale=True)
title = '<h3 align="center" style="font-size:20px"><b>Senegal Map</b></h3>'
title_html = folium.Html(title, script=True)
title_popup = folium.Popup(title_html, max_width=2650)

# Define the GeoJSON file paths
resting_points_file = '/content/drive/MyDrive/IKON/data/export_Resting-point.geojson'
corridors = '/content/drive/MyDrive/IKON/data/export_Corridor_(All).geojson'
village_file = '/content/drive/MyDrive/IKON/data/export_Village.geojson'
water_points_file = '/content/drive/MyDrive/IKON/data/export_Water-point_(All).geojson'

# Load the resting points GeoDataFrame
resting_points_gdf = gpd.read_file(resting_points_file)

# Load the village, water points, and corridor GeoDataFrames
corridor_gdf = gpd.read_file(corridors)
village_gdf = gpd.read_file(village_file)
water_points_gdf = gpd.read_file(water_points_file)



# Add Ferlo region as a GeoJSON layer with a custom color
ferlo_region = gpd.read_file('/content/drive/MyDrive/IKON/data/level4_regions_ikon.geojson')
ferlo_geojson = folium.GeoJson(ferlo_region, name='Ferlo Region', style_function=lambda x: {
    'fillColor': '#DEFCF9',  # Change the color here
    'color': '#DEFCF9',
    'weight': 1,
    'fillOpacity': 0.6
}).add_to(m)

# Add corridors as MultiLineString with a name for the legend and a custom color
corridor_json = corridor_gdf.to_json()
folium.features.GeoJson(corridor_json, name='Corridors', style_function=lambda x: {
    'color': '#16FF00',  # Change the color here
    'weight': 2
}).add_to(m)


# Add villages as small yellow circles with a name for the legend
for _, row in village_gdf.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=3,
        color='#F1C93B',
        fill=True,
        fill_color='#F1C93B',
        popup=row['name']  # Information to display on click
    ).add_to(m)

# Add resting points as small green circles on top of the corridors with a name for the legend
for _, row in resting_points_gdf.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=3,
        color='#EA1179',
        fill=True,
        fill_color='#EA1179',
        popup=row['name']  # Information to display on click
    ).add_to(m)

# Add water points as small blue circles with a name for the legend
for _, row in water_points_gdf.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=3,
        color='#0E21A0',
        fill=True,
        fill_color='#0E21A0',
        popup=row['name']  # Information to display on click
    ).add_to(m)

# Add radio information from the GeoDataFrame as location markers
for _, row in radio_gdf.iterrows():
    if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            tooltip=row['Radios'],
            popup=row['Radios'],  # Radio information to display on click
            icon=folium.Icon(icon='cloud', color='#D2DE32')
        ).add_to(m)

# Create a legend for the layers
legend_html = '''
<div style="position: fixed;
             bottom: 30%; left: 30%; transform: translate(-50%, -50%);
             width: 180px; height: 150px;
             border:2px solid grey; z-index:9999; font-size:14px;
             background-color: white;
             opacity: 0.8;">
    <b>Legend</b><br>
    <i style="background: #451952; width: 10px; height: 10px; display: inline-block;"></i> Resting Points<br>
    <i style="background: #F1C93B; width: 10px; height: 10px; display: inline-block;"></i> Villages<br>
    <i style="background: #0E21A0; width: 10px; height: 10px; display: inline-block;"></i> Water Points<br>
    <i style="background: #D2DE32; width: 10px; height: 10px; display: inline-block;"></i> Radio stations <br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))


# Add a layer control to the map
folium.LayerControl().add_to(m)

# Add the title to the map
title_popup.add_to(m)

# Save the map to an HTML file
m.save('Ikon_deployment_map_option_two.html')
m

## Wendou Data

In [ ]:
wendou_data = pd.read_excel('/content/drive/MyDrive/IKON/data/Historical_water_point_for_water_coverage.xlsx')

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geocoder")

# List of place names
place_names = list(wendou_data.Pond)

# Create lists to store latitude and longitude
latitudes = []
longitudes = []

# Geocode the place names and update the latitude and longitude lists
for place_name in place_names:
    try:
        location = geolocator.geocode(place_name)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)

# Append latitude and longitude columns to the DataFrame
wendou_data['Latitude'] = latitudes
wendou_data['Longitude'] = longitudes


# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(wendou_data.Longitude, wendou_data.Latitude)]
wendou_data_gdf = gpd.GeoDataFrame(wendou_data, geometry=geometry)
subset_wendou = wendou_data_gdf[~wendou_data_gdf['Latitude'].isnull()]

In [ ]:
subset_wendou.head()

In [ ]:

# Identify the column to compare
column_to_compare = 'Pond'

# Merge the dataframes on the specified column with an inner join
merged_df = wendou_ponds.merge(subset_wendou, on='Pond', how='inner').drop_duplicates(subset='Pond')

# Check if there are shared values
if not merged_df.empty:
    print("Shared values found in the column:", column_to_compare)
    print(merged_df[column_to_compare])
    #print(merged_df)
else:
    print("No shared values in the column:", column_to_compare)


In [ ]:
merged_df

In [ ]:
wendou_data_gdf.to_csv('/content/drive/MyDrive/IKON/wendou_data_gdf.csv')

## CHIRPS data

In [ ]:
import bs4
import requests
import zipfile
import gzip
import shutil

import numpy as np
import cv2
import os


## Downloading the data

In [ ]:
years = [x for x in range(2017,2023)]
for year in years:
    url = "http://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/" + str(year) + "/"
    r = requests.get(url)
    data = bs4.BeautifulSoup(r.text, "html.parser") #get page's html content
    failed = []
    for link in data.find_all("a")[5:]:
        file_name = link["href"]
        response = requests.get(url + file_name)
        if response.status_code==200:
            try:
                with open("/content/drive/MyDrive/IKON/chirps_data/" + file_name, 'wb') as f:
                    f.write(response.content)

                path = "/content/drive/MyDrive/IKON/chirps_data/" + file_name
                extracted_path = "/content/drive/MyDrive/IKON/chirps_data/Extracted/" + file_name[:-3]
                try:
                    with gzip.open(path, 'rb') as f_in:
                        with open(extracted_path, 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)
                except:
                    print("failed to extract ", file_name)

            except:
                print("failed to open link: ", file_name)
        else:
            failed.append(file_name)
            print(file_name, " download failed, moving to the next file...")

    IMG_DIR = '/content/drive/MyDrive/IKON/chirps_data'

In [ ]:
#sample data
wendou_ponds = gpd.read_file('/content/drive/MyDrive/IKON/output/wendou_ponds.shp')
two_ponds= wendou_ponds[wendou_ponds['Region']=='DIOURBEL']
wendou_ponds.head()

### Extract the daily rainfall values

In [ ]:
import glob
os.chdir('/content/drive/MyDrive/IKON/chirps_data/Extracted')
files = glob.glob("*.tif")

files = glob.glob("*.tif")


for j in range(len(two_ponds)):
    name = two_ponds.iloc[j,1]
    a = datetime.datetime.now()
    precipitation = []
    Date = []
    data = pd.DataFrame()
    lon,lat = two_ponds.iloc[j,3],two_ponds.iloc[j,2] #(x,y) x = lon, y = lat

    for i in range(len(files)):
        image = rasterio.open(files[i]) #import the image
        #precise location (point)
        # Row and column index of the lat and lon in the imagery
        row,col = image.index(lon,lat)
        #Extract rainfall
        precip_array = image.read(1) # read the rain band
        precipitation.append(precip_array[(row),(col)]) #precipitation
        Date.append(files[i][12:][:-4]) #Date
    data["Date"] = Date
    data["Rainfall mm"] = precipitation
    data.to_csv("Pond_"+ str(name) +"_chirps_data_2017_2022"+".csv", index = False)
    b = datetime.datetime.now()
    print("Pond_"+ str(name) +"_chirps_data_2017_2022"+".csv")
    print(b-a)
    print(b)


## Evaluating the chirps data

We are comparing the two ponds in DIOURBEL data sources to determine what source to use for automated downloading.

In [ ]:
# Function to convert date ranges to the last date of the range and extract the year
def convert_to_date_and_year(date_range):
    parts = date_range.split()
    start_day, end_day, month, year = parts[0].split('-')[0], parts[0].split('-')[1], parts[1], parts[2]
    #print(start_day, end_day, month, year )

    month_num = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }

    # Convert the month abbreviation to a numeric month
    month_numeric = month_num[month]

    # Create a date string for the last day of the range in YYYY-MM-DD format
    date_str = f"{end_day.zfill(2)}-{month_numeric}-{year}"
    #print(date_str)
    # Convert the date string to a datetime object
    date = pd.to_datetime(date_str,format="%d-%m-%Y")
    #print(date)

    return date, int(year)

#dogade pond
dogade = pd.read_csv('/content/drive/MyDrive/IKON/chirps_data/Dogade_1-Oct-2020_to_31-Aug-2021.csv')
# Apply the conversion function to the "Time" column
dogade[["Date", "Year"]] = dogade["Time"].apply(convert_to_date_and_year).apply(pd.Series)

# Drop the original "Time" column
dogade.drop(columns=["Time"], inplace=True)
dogade.columns = ['Rainfall mm', 'Date','Year']
dogade = dogade[['Date','Rainfall mm', 'Year']]

# Read the CSV file for Dogade_chirps_data
dogade_ch = pd.read_csv('/content/drive/MyDrive/IKON/chirps_data/Extracted/Pond_Dogade_chirps_data_2017_2022.csv')
# Extract the year from the 'Date' column and remove the last digit
dogade_ch['Year'] = dogade_ch['Date'].apply(lambda x: int(x.split('.')[0]))
# Convert the 'Date' column to a datetime format
dogade_ch['Date'] = pd.to_datetime(dogade_ch['Date'])

# Read the daily rainfall data
daily_rainfall_data = dogade_ch.copy() # Replace with the actual filename
#daily_rainfall_data['source'] = 'chirps'

# Read the second dataset with cumulative rainfall
cumulative_rainfall_data = dogade.copy() # Replace with the actual filename
cumulative_rainfall_data['source website'] = 'IRI'
daily_rainfall_data['Date'] = pd.to_datetime(daily_rainfall_data['Date'],format="%Y-%m-%d")
# Set the "Date" column as the index for both datasets
daily_rainfall_data.set_index('Date', inplace=True)
cumulative_rainfall_data.set_index('Date', inplace=True)

# Create an empty DataFrame to store the calculated cumulative rainfall
calculated_cumulative_rainfall = pd.DataFrame(columns=['Date', 'Rainfall mm', 'Year'])

# Iterate through the dates from the second dataset and calculate cumulative rainfall
for date in cumulative_rainfall_data.index:
    # Calculate the cumulative rainfall for the date in the daily rainfall data
    cumulative_rainfall = daily_rainfall_data.loc[date - pd.DateOffset(days=9):date]['Rainfall mm'].sum()

    # Append the calculated cumulative rainfall to the new DataFrame
    calculated_cumulative_rainfall = pd.concat([calculated_cumulative_rainfall, pd.DataFrame({'Date': [date], 'Rainfall mm': [cumulative_rainfall],
                                                                                              'Year': [cumulative_rainfall_data.loc[date]['Year']]})],
                                                ignore_index=True)


# Reset the index of the new DataFrame
calculated_cumulative_rainfall.reset_index(drop=True, inplace=True)
calculated_cumulative_rainfall['source website'] = 'chirps'
cumulative_rainfall_data.reset_index(inplace=True)
combined_data = pd.concat([calculated_cumulative_rainfall, cumulative_rainfall_data],axis=0)

fig, ax = plt.subplots(figsize = (10,6))
sns.lineplot(data = combined_data,ax=ax,
              x = 'Date',
             y = "Rainfall mm",
            hue = "source website",
            errorbar=None,
            sort= True, legend='auto').set_title('');
ax.xaxis.set_major_locator(md.MonthLocator(interval=1))
plt.gcf().autofmt_xdate()
plt.title("Comparison of Dogade pond rainfall in mm from two websites")
plt.grid(True)

In [ ]:
#nguilo pond
nguilo = pd.read_csv('/content/drive/MyDrive/IKON/chirps_data/Nguilo_1-Oct-2020_to_31-Aug-2021.csv')
# Apply the conversion function to the "Time" column
nguilo[["Date", "Year"]] = nguilo["Time"].apply(convert_to_date_and_year).apply(pd.Series)

# Drop the original "Time" column
nguilo.drop(columns=["Time"], inplace=True)
nguilo.columns = ['Rainfall mm', 'Date','Year']
nguilo = nguilo[['Date','Rainfall mm', 'Year']]


#nguilo pond
nguilo_ch = pd.read_csv('/content/drive/MyDrive/IKON/chirps_data/Extracted/Pond_Nguilo_chirps_data_2017_2022.csv')
# Extract the year from the 'Date' column and remove the last digit
nguilo_ch['Year'] = nguilo_ch['Date'].apply(lambda x: int(x.split('.')[0]))
# Convert the 'Date' column to a datetime format
nguilo_ch['Date'] = pd.to_datetime(nguilo_ch['Date'])

# Read the daily rainfall data
daily_rainfall_data = nguilo_ch.copy() # Replace with the actual filename
#daily_rainfall_data['source'] = 'chirps'

# Read the second dataset with cumulative rainfall
cumulative_rainfall_data = nguilo.copy() # Replace with the actual filename
cumulative_rainfall_data['source website'] = 'IRI'
daily_rainfall_data['Date'] = pd.to_datetime(daily_rainfall_data['Date'],format="%Y-%m-%d")
# Set the "Date" column as the index for both datasets
daily_rainfall_data.set_index('Date', inplace=True)
cumulative_rainfall_data.set_index('Date', inplace=True)

# Create an empty DataFrame to store the calculated cumulative rainfall
calculated_cumulative_rainfall_nguilo = pd.DataFrame(columns=['Date', 'Rainfall mm', 'Year'])

# Iterate through the dates from the second dataset and calculate cumulative rainfall
for date in cumulative_rainfall_data.index:
    # Calculate the cumulative rainfall for the date in the daily rainfall data
    cumulative_rainfall = daily_rainfall_data.loc[date - pd.DateOffset(days=9):date]['Rainfall mm'].sum()

    # Append the calculated cumulative rainfall to the new DataFrame
    calculated_cumulative_rainfall_nguilo = pd.concat([calculated_cumulative_rainfall_nguilo, pd.DataFrame({'Date': [date], 'Rainfall mm': [cumulative_rainfall],
                                                                                              'Year': [cumulative_rainfall_data.loc[date]['Year']]})],
                                                ignore_index=True)


# Reset the index of the new DataFrame
calculated_cumulative_rainfall_nguilo .reset_index(drop=True, inplace=True)
calculated_cumulative_rainfall_nguilo ['source website'] = 'chirps'
cumulative_rainfall_data.reset_index(inplace=True)
combined_data = pd.concat([calculated_cumulative_rainfall_nguilo, cumulative_rainfall_data],axis=0)
fig, ax = plt.subplots(figsize = (10,6))
sns.lineplot(data = combined_data,ax=ax,
              x = 'Date',
             y = "Rainfall mm",
            hue = "source website",
            errorbar=None,
            sort= True, legend='auto').set_title('');
ax.xaxis.set_major_locator(md.MonthLocator(interval=1))
plt.gcf().autofmt_xdate()
plt.title("Comparison of Nguilo pond rainfall in mm from two websites")
plt.grid(True)

### **Note**: The values obtained from the two websites are nearly identical, with a 99% correlation between the datasets. For the Chirps website we have an automated process for downloading and extracting the data, while the IRI website requires a manual download. Given this analysis, it is evident that the Chirps website is a more convenient and efficient alternative source for rainfall data.